In [3]:
# Installations
%%capture
!pip install pandas numpy scikit-learn statsmodels networkx matplotlib seaborn torch numba


##**casual_rules.py**

In [8]:
%%writefile causal_rules.pyreason
# Basic PyReason rules

# Example Rule 1: If it rains, the ground is wet.
@context(f'{t} and not {t-1}')
@forall(a, b)
def rule1(a: entity, b: entity):
    if a in Rains and b in Ground:
        return Wet(b)

# Example Rule 2: If the ground is wet and the sun is shining, it might steam.
@context(f'{t} and {t-1}')
@forall(a)
def rule2(a: entity):
    if Wet(a) and SunShining():
        return Steam(a) @ 0.7

# Example Rule 3: If plants are not watered, they may wither.
@context(f'{t}')
@forall(p)
def rule3(p: entity):
    if p in Plants and NotWatered(p):
        return Wither(p) @ 0.9

# Example Rule 4: If homework is not done, there may be trouble.
@context(f'{t}')
@forall(s)
def rule4(s: entity):
    if s in Students and NotDone(Homework(s)):
        return Trouble(s)

# Example Rule 5: If you sleep late, you feel tired the next day.
@context(f'{t} and {t+1}')
@forall(p)
def rule5(p: entity):
    if p in Persons and SleepLate(p):
        return Tired(p)

# Example Rule 6: If you eat healthy, you gain energy.
@context(f'{t}')
@forall(p)
def rule6(p: entity):
    if p in Persons and EatHealthy(p):
        return Energy(p)



Overwriting causal_rules.pyreason


In [9]:
import torch
import torch.nn as nn
import numpy as np
from numba import jit
import time

# -------------------------------
# Step 1: Simulated LLM Parser
# -------------------------------
def simulate_llm_parse(user_input: str) -> dict:
    """
    Expanded LLM parser for natural 'what if' and causal queries.
    """
    user_input_lower = user_input.lower()

    # Handle "what if" queries
    if user_input_lower.startswith("what if"):
        cause = user_input_lower.replace("what if", "").strip("?!.")
        return {
            "status": "parsed",
            "query_type": "causal_inference",
            "entities": [
                {"type": "cause", "name": cause},
                {"type": "effect", "name": "outcome"} # generic effect placeholder
            ]
        }

    # Handle "causal effect of" or "what happens if"
    if "causal effect of" in user_input_lower or "what happens if" in user_input_lower:
        entities = []
        if "on" in user_input_lower:
            parts = user_input_lower.split("on")
            if len(parts) == 2:
                cause = parts[0].replace("causal effect of", "").replace("what happens if", "").strip()
                effect = parts[1].strip()
                if cause:
                    entities.append({"type": "cause", "name": cause})
                if effect:
                    entities.append({"type": "effect", "name": effect})
        return {"status": "parsed", "query_type": "causal_inference", "entities": entities}

    # Handle definition queries
    if "define" in user_input_lower or "what is" in user_input_lower:
        definition_topic = user_input_lower.replace("define", "").replace("what is", "").strip()
        return {"status": "parsed", "query_type": "definition", "topic": definition_topic}

    # Handle rule listing
    if "list rules" in user_input_lower:
        return {"status": "parsed", "query_type": "list_rules"}

    return {"status": "unparsable", "original_input": user_input}


# -------------------------------
# Step 2: Convert Parsed Input
# -------------------------------
def convert_to_pyreason_input(parsed_input: dict) -> str or None:
    """
    Converts the structured dictionary from the simulated LLM into a format
    that PyReason can use.
    """
    if parsed_input.get("status") == "parsed":
        query_type = parsed_input.get("query_type")

        if query_type == "causal_inference":
            entities = parsed_input.get("entities", [])
            cause = next((e["name"] for e in entities if e["type"] == "cause"), None)
            effect = next((e["name"] for e in entities if e["type"] == "effect"), None)
            if cause and effect:
                return f"query effect of {cause} on {effect}"
            else:
                return None

        elif query_type == "definition":
            topic = parsed_input.get("topic")
            if topic:
                return f"define {topic}"
            else:
                return None

        elif query_type == "list_rules":
            return "list rules"

        else:
            return None

    elif parsed_input.get("status") == "unparsable":
        return None

    else:
        return None


# -------------------------------
# Step 3: Simple Reasoning Engine
# -------------------------------
def reason_with_rules(pyreason_input: str) -> str:
    """
    Very basic rule matching. In real PyReason, you'd load and evaluate rules.
    """
    rules = {
        "not watering plants": "plants may wither",
        "not doing homework": "you might get in trouble",
        "rain": "the ground becomes wet",
        "ground is wet and sun is shining": "it might steam"
    }

    for cause, outcome in rules.items():
        if cause in pyreason_input:
            return f"If {cause}, then {outcome}."
    return "I don't have a rule for that yet."


# -------------------------------
# Step 4: NN + Numba Examples
# -------------------------------
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


def predict_outcome_with_nn(input_data):
    input_dim = input_data.shape[1]
    hidden_dim = 5
    output_dim = 1
    model = SimpleNN(input_dim, hidden_dim, output_dim)
    input_tensor = torch.tensor(input_data, dtype=torch.float32)
    output_probability = model(input_tensor)
    return output_probability.detach().numpy()


@jit(nopython=True)
def fast_array_operation(arr):
    result = np.zeros_like(arr)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            result[i, j] = arr[i, j] * 2.0 + 1.0
    return result


# -------------------------------
# Step 5: Chatbot Loop
# -------------------------------
def main_chatbot_loop():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Chatbot: Goodbye!")
            break

        parsed_input = simulate_llm_parse(user_input)
        pyreason_input = convert_to_pyreason_input(parsed_input)

        if pyreason_input:
            response = reason_with_rules(pyreason_input)
            print(f"Chatbot: {response}")
        else:
            print("Chatbot: I couldn't understand that. Can you rephrase?")


# Example quick tests
if __name__ == "__main__":
    dummy_input = np.array([[0.5, 0.1, 0.9]])
    prediction = predict_outcome_with_nn(dummy_input)
    print(f"NN Prediction: {prediction}")

    large_array = np.random.rand(500, 500)
    start_time = time.time()
    result_numpy = large_array * 2.0 + 1.0
    print(f"NumPy op time: {time.time() - start_time:.6f} sec")

    start_time = time.time()
    result_numba = fast_array_operation(large_array)
    print(f"Numba op time: {time.time() - start_time:.6f} sec")

    # Start chatbot
    main_chatbot_loop()


NN Prediction: [[0.41763815]]
NumPy op time: 0.000827 sec
Numba op time: 0.194753 sec
You: what if it rains?
Chatbot: If rain, then the ground becomes wet.
You: what if i don't plants are not watered?
Chatbot: I don't have a rule for that yet.
You: what if i sleep late?
Chatbot: I don't have a rule for that yet.
You: what if you sleep late?
Chatbot: I don't have a rule for that yet.
You: quit
Chatbot: Goodbye!
